In [ ]:
%matplotlib inline

# Normal modules
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_curve, roc_auc_score
# Torch modules
import torch
from torch import nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import models
from torchvision import transforms

# Own codes written for this tutorial
from codes import train_utils
from codes import val_utils
from codes.augmentations import *
from codes.dataset import InvasiveSpeciesDataset


In [ ]:
def plot_roc(gt, preds):
    fpr1, tpr1, _ = roc_curve(gt, preds)
    plt.figure(figsize=(6,6))
    plt.plot(fpr1, tpr1,lw=2, c='b')
    plt.plot([0, 1], [0,1], '--',c='black')
    plt.grid()
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.tight_layout()
    plt.show()

### Parameters

In [ ]:
SEED = 42
LR = 1e-2
BS = 8
DATSET_LOC = 'data/train/'
WD = 1e-4
MAX_EPOCH = 5

## Exploration

In [ ]:
metadata = pd.read_csv('data/train_labels.csv')
metadata.name = metadata.apply(lambda x: str(x[0]), 1)
metadata.head()

In [ ]:
resize = (128,128)

fname = str(metadata.name.loc[2]) + '.jpg'
fname = os.path.join(DATSET_LOC, fname)

img = cv2.imread(fname)
img = cv2.resize(img, resize)
img_inv = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


fname = str(metadata.name.loc[0]) + '.jpg'
fname = os.path.join(DATSET_LOC, fname)

img = cv2.imread(fname)
img = cv2.resize(img, resize)
img_none = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(121)
plt.title('None')
plt.imshow(img_none)
plt.subplot(122)
plt.title('Ivasive')
plt.imshow(img_inv)
plt.show()

### Train/Val Splits

In [ ]:
sss = StratifiedShuffleSplit(2, train_size=0.8, test_size=0.2, random_state=SEED)
train_index, test_index = next(sss.split(metadata['name'], metadata['invasive']))

In [ ]:
X_train = metadata.iloc[train_index]
X_val = metadata.iloc[test_index]

### Augmentations and transforms

In [ ]:
train_transforms  = transforms.Compose([
    lambda x: cv2.resize(x, (130, 130)),
    lambda x: augment_random_linear(x, sr=10, ssx=0.2, ssy=0.2),
    lambda x: augment_random_crop(x, (128, 128)),
    lambda x: augment_random_flip(x, hprob=0.5, vprob=0),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms  = transforms.Compose([
    lambda x: cv2.resize(x, (130, 130)),
    lambda x: center_crop(x, (128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Data loaders 

In [ ]:
train_ds = InvasiveSpeciesDataset(DATSET_LOC, X_train, train_transforms)
train_loader = data.DataLoader(train_ds, BS, shuffle=True, drop_last=True, num_workers=4)

val_ds = InvasiveSpeciesDataset(DATSET_LOC, X_val, val_transforms)
val_loader = data.DataLoader(val_ds, BS, shuffle=True, drop_last=True, num_workers=4)

### Network definition

In [ ]:
class SqueezeFineTune(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = models.squeezenet1_1(pretrained=True).features
        self.classifier = nn.Sequential(nn.Dropout(0.5),
                                        nn.Linear(512,1)
                                       )
    def forward(self, x):
        o = self.features(x)
        o = F.adaptive_avg_pool2d(o, 1)
        o = o.view(o.size(0), -1)
        o = self.classifier(o)
        return o

### Network, optimizer, loss

In [ ]:
# Initializing the network
net = SqueezeFineTune()
# Creating the optimizer
optimizer = optim.Adam(net.classifier.parameters(), lr=LR, weight_decay=WD)
# Creating the optimizer
criterion = nn.BCEWithLogitsLoss()

In [ ]:
train_losses = []
val_losses = []

for epoch in range(MAX_EPOCH):
    # Training
    train_loss = train_utils.train_epoch(epoch, net, optimizer, train_loader, criterion, MAX_EPOCH)
    # Validating
    val_loss, preds_val, truth_val = val_utils.validate_epoch(net, val_loader, criterion)
    auc_val = roc_auc_score(truth_val, preds_val)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print('Epoch {} | Train loss: {:.4f} | Val. loss {:.4f} | Val. AUC {:.4f}'.format(epoch + 1, train_loss, val_loss, auc_val))

In [ ]:
plot_roc(truth_val, preds_val)
roc_auc_score(truth_val, preds_val)